In [1]:
from googleapiclient.discovery import build
import os

In [2]:
from keys import api_key, api_key2

In [3]:
from datetime import datetime, timedelta, timezone
import isodate

In [4]:
from datetime import datetime, timedelta
import pytz

In [5]:
import pytz

In [6]:
youtube = build('youtube', 'v3', developerKey=api_key2)

In [7]:
def get_latest_video_id(channel_id):
    # Retrieve the list of videos uploaded to the channel.
    request = youtube.search().list(
        part="snippet",
        channelId=channel_id,
        maxResults=50,  # Increase the maxResults to retrieve more videos
        order="date",  # Order by date to get the latest video
        type="video"  # Look for videos only (no playlists or channels)
    )

    response = request.execute()

    for item in response.get('items', []):
        video_id = item['id']['videoId']

        # Make a request to get the video details
        video_request = youtube.videos().list(
            part="snippet,liveStreamingDetails,contentDetails",
            id=video_id
        )
        video_response = video_request.execute()

        video = video_response['items'][0]

        # Check if the video is not a livestream
        if 'liveStreamingDetails' not in video:
            # Get the video duration
            duration = video['contentDetails']['duration']
            # Convert the duration to seconds
            duration_seconds = isodate.parse_duration(duration).total_seconds()

            # Check if the video duration is greater than 60 seconds (not a YouTube Short)
            if duration_seconds > 70:
                video_published_at = datetime.strptime(item['snippet']['publishedAt'], '%Y-%m-%dT%H:%M:%SZ')
                video_published_at = video_published_at.replace(tzinfo=pytz.UTC)
                now = datetime.now(pytz.UTC)

                # Check if the video is not older than 1 day
                if now - video_published_at <= timedelta(days=1):
                    return video_id

    return None

def get_latest_video_ids(video_ids, channel_ids):
    # Loop over channel IDs and append the video IDs to the list
    for channel_id in channel_ids:
        video_id = get_latest_video_id(channel_id)
        if video_id:
            video_ids.append(video_id)
            print(f"Added Video ID: {video_id}")

# Example usage:
channel_ids = ['UCvJZEG5x-DVYZKTz--pS39w', 'UC7QHSNOKZvbqpXWFv0SiYDQ',"UCinBfsRWiOlzxg7DvjZ5mdQ", "UCUP_ao_7-Yct5FcVIA4Kobg", "UCAHJqpJPqqLLi7siVQpD-3A" , 'UCfdPOTevbfCh_QHsyPeZ8MQ',
               "UCYKtr6GfycBqQJf32tbQSbQ", 'UCpBPTKot6Im9SWs1upQMZKA', 'UCTeFsS-bP0XEt3NBMjfW2cA', 'UC-m6zNItyoDk5lSykDlhE4Q' , 'UCmlfS56nZsnX_5F1fjvX0QA', 'UCoAuxtwgh8LSHeMFqEwjgHA']  # List of example channel IDs
latest_video_ids = []
get_latest_video_ids(latest_video_ids, channel_ids)

print("Latest video IDs:", latest_video_ids)

Added Video ID: 3HBfq2oT07w
Added Video ID: MMz0vSVvSSg
Added Video ID: i8J_XaCuzvE
Added Video ID: n1TfQBnCocI
Added Video ID: wdZvTOHocxU
Added Video ID: vDHrasfFtQQ
Added Video ID: DTDH1-_-btk
Added Video ID: nuIK0CpyyEg
Latest video IDs: ['3HBfq2oT07w', 'MMz0vSVvSSg', 'i8J_XaCuzvE', 'n1TfQBnCocI', 'wdZvTOHocxU', 'vDHrasfFtQQ', 'DTDH1-_-btk', 'nuIK0CpyyEg']


In [8]:
latest_video_ids

['3HBfq2oT07w',
 'MMz0vSVvSSg',
 'i8J_XaCuzvE',
 'n1TfQBnCocI',
 'wdZvTOHocxU',
 'vDHrasfFtQQ',
 'DTDH1-_-btk',
 'nuIK0CpyyEg']

In [9]:
import youtube_dl

def get_latest_video_url(channel_urls):
    video_urls = []
    for channel_url in channel_urls:
        with youtube_dl.YoutubeDL({'quiet': True}) as ydl:
            result = ydl.extract_info(channel_url, download=False)
            if 'entries' in result:
                # Sort videos by upload date
                latest_video = sorted(result['entries'], key=lambda x: x['upload_date'], reverse=True)[0]
                video_urls.append(latest_video['webpage_url'])
    return video_urls


In [10]:
import feedparser
from datetime import datetime, timedelta

# Function to get the latest video ID and upload date from a YouTube channel using RSS feed
def get_latest_video_id(channel_id):
    # Construct the RSS feed URL for the YouTube channel
    feed_url = f"https://www.youtube.com/feeds/videos.xml?channel_id={channel_id}"
    # Parse the feed
    feed = feedparser.parse(feed_url)
    # Get the first entry, which is the latest video
    latest_video = feed.entries[0]
    # Parse the published date
    published_date = datetime.strptime(latest_video.published, '%Y-%m-%dT%H:%M:%S+00:00')
    # Check if the video is less than a day old
    if datetime.utcnow() - published_date <= timedelta(days=2):
        # Extract video ID from the URL
        video_id = latest_video.link.split("=")[-1]
        # Return the video ID if the video is within one day old
        return video_id
    else:
        return None


def get_latest_video_name(channel_id):
    # Construct the RSS feed URL for the YouTube channel
    feed_url = f"https://www.youtube.com/feeds/videos.xml?channel_id={channel_id}"
    # Parse the feed
    feed = feedparser.parse(feed_url)
    # Get the first entry, which is the latest video
    latest_video = feed.entries[0]
    # Parse the published date
    published_date = datetime.strptime(latest_video.published, '%Y-%m-%dT%H:%M:%S+00:00')
    # Check if the video is less than a day old
    if datetime.utcnow() - published_date <= timedelta(days=2):
        # Extract video ID from the URL
        title= latest_video.title
        # Return the video ID if the video is within one day old
        return title
    else:
        return None

In [11]:
latest_video_ids

['3HBfq2oT07w',
 'MMz0vSVvSSg',
 'i8J_XaCuzvE',
 'n1TfQBnCocI',
 'wdZvTOHocxU',
 'vDHrasfFtQQ',
 'DTDH1-_-btk',
 'nuIK0CpyyEg']

In [12]:
from youtube_transcript_api import YouTubeTranscriptApi


def get_youtube_captions(video_id):
    try:
        # Retrieve the available subtitles
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        current_minute = 0
        paragraph = []
        paragraphs = []
        
        # Process each subtitle line
        for item in transcript:
            # Calculate the minute mark of the current subtitle
            item_minute = int(item['start'] // 60)
            
            # Check if the current subtitle still falls within the current minute window
            if item_minute == current_minute:
                paragraph.append(item['text'])
            else:
                # Store the completed paragraph and start a new one
                paragraphs.append(' '.join(paragraph))
                paragraph = [item['text']]
                current_minute = item_minute
        
        # Add the last paragraph if any
        if paragraph:
            paragraphs.append(' '.join(paragraph))
        
        # Join all paragraphs with two newlines
        formatted_text = '\n\n'.join(paragraphs)
        
        # Optionally save the subtitles to a file
        with open(f'{video_id}_subtitles.txt', 'w', encoding='utf-8') as f:
            f.write(formatted_text)
        
        return formatted_text
    except Exception as e:
        return str(e)

# Assuming latest_video_ids is a list of video IDs
for video_id in latest_video_ids:
    captions = get_youtube_captions(video_id)
    print(captions)

Stanley dren Miller is considered one of the greatest investors and Traders of all time so when he cuts his stake in Nvidia we need to be paying attention what is it that Stanley seeing that we don't just yet and also with commodities going up as markets increase for the first time since the 1980s should we be worried well the secret might be in actually what's happened over the last 3 days and in today's video we'll look at the next weeks months and year for markets around the world including the US dollar and Tesla's fall back to one of the most traded zones join us as always as we cover stocks Commodities and cryptos together for the days ahead let's get into it this will be a good one well welcome back everybody to The Daily Show my name is Tom and as always we'll be going through the latest macro data and of course leading indicators that matter to you as investors and Traders if you want to better understand why markets move and where the flows are going then remember subscribe a

In [13]:
'Analyze the content from the file I've provided, to answer: What is the short term forecast of the market, especially SPY and ETFs. Give me the price ranges, bullish cases or indicators, and bearish cases or indicators.'



SyntaxError: unterminated string literal (detected at line 1) (931488424.py, line 1)

In [ ]:
'Analyze the content from the file I've provided, to answer: What is the short term forecast of the market.'

In [ ]:
'Outline the bullish and bearish cases. The amount of bullet points should reflect the analyst's stance on whether they are bullish or bearish overall. If there are any specific symbols and price levels to look out for, state them.'

In [ ]:
"Give me a summary and investment advice based on this analysis."

In [ ]:
import openai

def get_chatgpt_response(prompt_file_path):
    # Load your API key from an environment variable or direct string (not recommended for production)
    openai.api_key = 'your-api-key-here'

    # Read the prompt from the file
    with open(prompt_file_path, 'r') as file:
        prompt = file.read()

    # Send the prompt to the ChatGPT model
    response = openai.ChatCompletion.create(
        model="gpt-4",  # or use "text-davinci-003" for GPT-3.5
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Return the response text
    return response.choices[0].message['content']

# Example usage
#prompt_path = 'path_to_your_prompt_file.txt'  # Change this to the path of your prompt file
#response_text = get_chatgpt_response(prompt_path)
#print("ChatGPT response:", response_text)